In [ ]:
from bs4 import BeautifulSoup
import requests

### Global Constants

In [ ]:
SEARCH_ENGINE_URL = "https://www.ask.com/web"
SEARCH_RESULT_SELECTOR = "PartialSearchResults-item-title-link result-link"
N_OF_RESULTS_REQUIRED = 10

QUERY = "How is the spinning mule fuelled"

### Fetch Results

In [ ]:
search_result = requests.get(SEARCH_ENGINE_URL, params= {
    "q": QUERY,
    "page": 1
})

In [ ]:
soup = BeautifulSoup(search_result.text, "html.parser")

In [ ]:
soup.title

In [ ]:
retrieved_results = soup.find_all("a", attrs= { "class": SEARCH_RESULT_SELECTOR }, limit=N_OF_RESULTS_REQUIRED)


In [ ]:
retrieved_results[0]

In [ ]:
type(retrieved_results[0])

In [ ]:
retrieved_results[0]["href"]

In [ ]:
type(retrieved_results[0]["href"])

In [ ]:
results = list()
for retrieved_result in retrieved_results:
    results.append(retrieved_result["href"])

results

### Check results count

In [ ]:
results_count = soup.find("div", attrs={"class": "PartialResultsHeader-summary"}).contents[0]
results_count

In [ ]:
results_count_stripped = results_count.replace(",", "")
results_count_stripped

In [ ]:
import re

prog = re.compile(r"\d+")

result = prog.findall(results_count_stripped)

if result is not None:
    counts = [int(r) for r in result]

In [ ]:
counts

In [ ]:
set([0, 1, 2, 2, 3, 4, 5, 6, 7, 7])

In [ ]:
dict.fromkeys([1,2 , 3, 4, 4, 5, 5, 6]).keys()

In [ ]:
from collections import OrderedDict

test = OrderedDict.fromkeys([1,2 , 3, 4, 4, 5, 5, 6])

list(test.keys()), len(test)

### Testing SearchEngine

In [ ]:
from search_engine import SearchEngine

In [ ]:
SearchEngine.search(SEARCH_ENGINE_URL, QUERY, { "class": SEARCH_RESULT_SELECTOR }, result_limit=10, should_sleep=False)

In [ ]:
from os import getcwd

getcwd()

In [ ]:
QUERIES_SET_PATH = "./../data/100QueriesSet3.txt"

from utils import read_queries

read_queries(QUERIES_SET_PATH, 96)

In [ ]:
from utils import write_checkpoint, read_checkpoint

# write_checkpoint("./checkpoint.txt", 10)
read_checkpoint("./checkpoint.txt")

In [ ]:
from utils import save_results
import logging

logging.basicConfig(format="%(asctime)-15s [%(levelname)s] %(funcName)s: %(message)s", level=logging.INFO)

save_results("./../submission/hw1.json", {'aaaaaaab': 1})

In [ ]:
from main import main

main()

In [ ]:
from collections import OrderedDict

o = dict()
o.update({"a": [1, 2, 3], "b": [2, 4, 5]})
o
o.update({"c": [4, 6, 7], "d": [8, 9, 10]})
o

### Scraping DuckDuckGo

In [ ]:
DUCKDUCKGO_URL = "https://html.duckduckgo.com/html"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36",
    "Content-Type": "application/x-www-form-urlencoded"
}

QUERY_STRING = "What are some cheats for pets on barbiegirlscom"


In [ ]:
from bs4 import BeautifulSoup
import requests

results_page = requests.get(DUCKDUCKGO_URL, params={"q": QUERY_STRING}, headers=HEADERS)

if results_page.status_code != 200:
    print("error scrapping duckduckgo")

soup = BeautifulSoup(results_page.text, "html.parser")


In [ ]:
raw_results = soup.find_all("a", attrs={"class": "result__a"})

raw_results

In [ ]:
raw_results[0]["href"]

In [ ]:
links = list()

for result in raw_results:
    links.append(result["href"])

links

In [ ]:
len(links)

In [ ]:
inputs = soup.find_all("div", attrs={"class": "nav-link"})[0].find("form").find_all("input", attrs={"type": "hidden"})
inputs

In [ ]:
data = dict()

for inpt in inputs:
    data[inpt["name"]] = inpt["value"]

data

In [ ]:
results_page = requests.post(DUCKDUCKGO_URL, data, headers=HEADERS)

if results_page.status_code != 200:
    print("error scrapping duckduckgo")

soup = BeautifulSoup(results_page.text, "html.parser")

raw_results = soup.find_all("a", attrs={"class": "result__a"})

links = list()

for result in raw_results:
    links.append(result["href"])

links


### Comparing with Google Results

In [ ]:
REFERENCE_SEARCH_RESULT_FILE_PATH = "./../data/Google_Result3.json"
OPPONENT_SEARCH_RESULTS_FILE_PATH = "./../submission/hw1.json"
OUTPUT_FILE_PATH = "./../submission/hw1.csv"

In [ ]:
from utils import load_results

reference_search_results = load_results(REFERENCE_SEARCH_RESULT_FILE_PATH)
opponent_search_results = load_results(OPPONENT_SEARCH_RESULTS_FILE_PATH)

In [ ]:
import pandas as pd
import numpy as np

# get first query
query = list(reference_search_results.keys())[1]
query

In [ ]:
def normalize_url(urls):
    # takes a list of URLs and returns processed list
    normalized_urls = list()
    for url in urls:
        url = url.replace("www.", "")
        url = url.replace("https://", "http://")
        url = url.strip("/")
        normalized_urls.append(url)
    
    return normalized_urls


In [ ]:
reference_set = normalize_url(reference_search_results[query])
opponent_set = normalize_url(opponent_search_results[query])


In [ ]:
overlap = len(set(reference_set).intersection(set(opponent_set)))
overlap

In [ ]:
spearman_correlation = list()
diff = list()

for idx, url in enumerate(reference_set):
    try:
        idx_2 = opponent_set.index(url)
        spearman_correlation.append((idx + 1, idx_2 + 1))
        diff.append(idx - idx_2)
    except:
        pass

spearman_correlation, diff

In [ ]:
rho = 1 - ((6 * np.sum(np.power(diff, 2))) / (overlap * (np.power(overlap, 2) - 1)))
rho

In [4]:
REFERENCE_SEARCH_RESULT_FILE_PATH = "./../data/Google_Result4.json"
OPPONENT_SEARCH_RESULTS_FILE_PATH = "./../submission/hw1_ddg.json"
OUTPUT_FILE_PATH = "./../submission/hw1.csv"


In [ ]:
from task2 import main

main(REFERENCE_SEARCH_RESULT_FILE_PATH, OPPONENT_SEARCH_RESULTS_FILE_PATH, OUTPUT_FILE_PATH)

In [5]:
from utils import load_results


reference_search_results = load_results(REFERENCE_SEARCH_RESULT_FILE_PATH)
opponent_search_results = load_results(OPPONENT_SEARCH_RESULTS_FILE_PATH)

len(reference_search_results), len(opponent_search_results)

2023-01-29 18:25:27,714 [INFO] load_results: Opening file ./../data/Google_Result4.json
2023-01-29 18:25:27,716 [INFO] load_results: Opening file ./../submission/hw1_ddg.json


(100, 100)

In [7]:
for ref_result in reference_search_results:
    print(ref_result)
    if ref_result not in opponent_search_results:
        print("NOT HERE!!!!!!!!!!!")
        break

In [ ]:
def can_search_further(soup):
    next_btn_div = soup.find_all("div", attrs={"class": "nav-link"})
    if next_btn_div is not None:
        return True
    else:
        return False


In [1]:
import logging
from duckduckgo import SearchEngine

logging.basicConfig(format="%(asctime)-15s [%(levelname)s] %(funcName)s: %(message)s", level=logging.INFO)

results = SearchEngine.search("https://html.duckduckgo.com/html/", "A two dollar bill from 1953 is worth what ", {"class": "result__a"})
results


2023-01-29 16:23:04,502 [INFO] search: Starting to get search results until 10 results obtained for query
2023-01-29 16:23:04,502 [INFO] search: Sleeping for 36 seconds
2023-01-29 16:23:40,507 [INFO] search: Search Page: 1
2023-01-29 16:23:40,507 [INFO] search: Now Searching: A two dollar bill from 1953 is worth what 
2023-01-29 16:23:41,365 [INFO] search: Requested: https://html.duckduckgo.com/html/?q=A%2Btwo%2Bdollar%2Bbill%2Bfrom%2B1953%2Bis%2Bworth%2Bwhat%2B&page=1
2023-01-29 16:23:41,365 [INFO] search: Search Engine Response Status Code: 200
2023-01-29 16:23:41,378 [INFO] search: Extracting result from search engine response
2023-01-29 16:23:41,378 [INFO] __scrape_search_result: Getting all result anchor tags
2023-01-29 16:23:41,379 [INFO] __scrape_search_result: Extracting the links (href) from result anchor tags
2023-01-29 16:23:41,380 [INFO] __scrape_search_result: Retrieved 10 links
2023-01-29 16:23:41,380 [INFO] search: No of results collected: 10
2023-01-29 16:23:41,380 [INF

OrderedDict([('https://www.silverrecyclers.com/blog/1953-2-dollar-bill.aspx',
              None),
             ('https://www.vipartfair.com/1953-2-dollar-bill-value/', None),
             ('https://treasurepursuits.com/1953-2-dollar-bill-value-whats-it-worth/',
              None),
             ('https://oldcurrencyvalues.com/1953_red_seal_two_dollar/', None),
             ('https://antiques.lovetoknow.com/antique-price-guides/how-determine-2-dollar-bill-values',
              None),
             ('https://jewelrymarquis.com/coin/1953-2-dollar-bill-value',
              None),
             ('https://www.thesprucecrafts.com/red-seal-two-dollar-bill-4782928',
              None),
             ('https://hobbylark.com/collecting/Value-of-Two-Dollar-Bills',
              None),
             ('https://www.pcgs.com/news/what-is-my-two-dollar-bill-worth',
              None),
             ('https://www.thesprucecrafts.com/two-dollar-bill-worth-4776868',
              None),
             ('htt

In [3]:
list(results.keys()

19